In [13]:
import pandas as pd
import numpy as np
import torch
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer,
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Используется Apple Silicon (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Используется CUDA GPU")
else:
    device = torch.device("cpu")
    print("Используется CPU")

print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

Используется Apple Silicon (MPS)
Device: mps
PyTorch version: 2.9.1


In [14]:
# === ЗАГРУЗКА ПОДГОТОВЛЕННЫХ ДАННЫХ ===

# Загружаем CSV файлы из preprocessing
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
test_df = pd.read_csv('test_data.csv')

print(f"\nДанные загружены:")
print(f"   Train: {len(train_df):,} пар")
print(f"   Val:   {len(val_df):,} пар")
print(f"   Test:  {len(test_df):,} пар")

# Примеры данных
print("\n=== ПРИМЕРЫ ДАННЫХ ===")
for i in range(3):
    print(f"\nПример {i+1}:")
    print(f"  INPUT:  {train_df.iloc[i]['input'][:80]}...")
    print(f"  OUTPUT: {train_df.iloc[i]['output']}")


Данные загружены:
   Train: 52,854 пар
   Val:   6,600 пар
   Test:  6,607 пар

=== ПРИМЕРЫ ДАННЫХ ===

Пример 1:
  INPUT:  Поехали два депутата городской думы на рыбалку. Порыбачили, сели выпить, покушат...
  OUTPUT: Нет, Миш. Совпадения, не совпадения, а новый морг я уже успел пообещать…

Пример 2:
  INPUT:  В баре трое охотников рассказывают друг другу о своих успехах на сафари. Первый:...
  OUTPUT: А на меня помчался буйвол с наставленными рогами, ну, я в него - бух!

Пример 3:
  INPUT:  Утром просыпаются муж и жена. Жена толкает мужа локтем в бок: - Дорогой, а ведь ...
  OUTPUT: Да? Ну и что?


In [16]:
# === ЗАГРУЗКА ТОКЕНИЗАТОРА И МОДЕЛИ ===
MODEL_NAME = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

# Настройка токенизатора
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Токенизатор загружен")
print(f"   Vocab size: {len(tokenizer):,}")
print(f"   PAD token: '{tokenizer.pad_token}'")

print(f"\nЗагрузка модели: {MODEL_NAME}")
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# Настройка модели
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

print("Модель загружена")
print(f"   Параметров: {model.num_parameters():,}")
print(f"   Размер: {model.num_parameters() * 4 / 1024 / 1024:.1f} MB")

Токенизатор загружен
   Vocab size: 50,257
   PAD token: '<pad>'

Загрузка модели: sberbank-ai/rugpt3small_based_on_gpt2
Модель загружена
   Параметров: 125,226,240
   Размер: 477.7 MB


In [17]:
# === ПОДГОТОВКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ ===

# Гиперпараметры токенизации (из preprocessing)
MAX_INPUT_LENGTH = 128
MAX_OUTPUT_LENGTH = 32

def format_prompt(input_text, output_text=None):
    """
    Форматирует промпт для диалоговой модели.
    Формат: <контекст> [SEP] <ответ> [EOS]
    """
    if output_text:
        # Для обучения: добавляем ответ
        prompt = f"{input_text}\n{output_text}{tokenizer.eos_token}"
    else:
        # Для инференса: только контекст
        prompt = f"{input_text}\n"
    return prompt

def tokenize_function(examples):
    """
    Токенизирует примеры для обучения.
    """
    prompts = [
        format_prompt(inp, out) 
        for inp, out in zip(examples['input'], examples['output'])
    ]
    
    # Токенизируем
    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=MAX_INPUT_LENGTH + MAX_OUTPUT_LENGTH,
        padding='max_length',
        return_tensors=None
    )
    
    # Для языковой модели labels = input_ids
    tokenized['labels'] = tokenized['input_ids'].copy()
    
    return tokenized

# Конвертируем в Dataset
print("\nКонвертация в HuggingFace Dataset...")
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print("\nТокенизация данных...")
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['input', 'output'],
    desc="Токенизация train"
)

val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['input', 'output'],
    desc="Токенизация val"
)

print(f"\nДанные подготовлены:")
print(f"   Train dataset: {len(train_dataset)} примеров")
print(f"   Val dataset: {len(val_dataset)} примеров")

# Пример токенизированных данных
print("\n=== ПРИМЕР ТОКЕНИЗИРОВАННЫХ ДАННЫХ ===")
example = train_dataset[0]
print(f"Input IDs shape: {len(example['input_ids'])}")
print(f"Decoded text: {tokenizer.decode(example['input_ids'])[:200]}...")


Конвертация в HuggingFace Dataset...

Токенизация данных...


Токенизация train:   0%|          | 0/52854 [00:00<?, ? examples/s]

Токенизация val:   0%|          | 0/6600 [00:00<?, ? examples/s]


Данные подготовлены:
   Train dataset: 52854 примеров
   Val dataset: 6600 примеров

=== ПРИМЕР ТОКЕНИЗИРОВАННЫХ ДАННЫХ ===
Input IDs shape: 160
Decoded text: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>Поехали два депутата городской думы на рыбалку. Порыбачили, сели выпить, покушать. - Петрович! Ты чего после аварии тако...


In [18]:
# === НАСТРОЙКА ОБУЧЕНИЯ ===

# Создаем директорию для сохранения модели
import os
os.makedirs("models", exist_ok=True)

training_args = TrainingArguments(
    # Основные параметры
    output_dir="./models",
    overwrite_output_dir=True,
    
    # Параметры обучения
    num_train_epochs=3,  # Немного эпох для baseline
    per_device_train_batch_size=8,  # Небольшой batch для M4 Pro
    per_device_eval_batch_size=8,
    
    # Оптимизация
    learning_rate=5e-5,  # Стандартный LR для файнтюнинга
    weight_decay=0.01,
    warmup_steps=500,
    
    # Логирование и сохранение
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,  # Храним только 2 последних чекпоинта
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=500,
    
    # Оптимизация для M4 Pro
    fp16=False,
    bf16=False,
    
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none",
    seed=42,
)

print("Параметры обучения настроены:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Warmup steps: {training_args.warmup_steps}")
print(f"   Total training steps: ~{len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs}")

Параметры обучения настроены:
   Epochs: 3
   Batch size: 8
   Learning rate: 5e-05
   Warmup steps: 500
   Total training steps: ~19818


In [19]:
# === СОЗДАНИЕ TRAINER ===
# Data collator для языковой модели
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Trainer создан")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Validation samples: {len(val_dataset)}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation steps: {training_args.gradient_accumulation_steps}")

Trainer создан
   Training samples: 52854
   Validation samples: 6600
   Batch size: 8
   Gradient accumulation steps: 1


In [20]:
# === ОБУЧЕНИЕ МОДЕЛИ ===
print("\nСтарт обучения...")

# Обучаем модель
train_result = trainer.train()

print("\n" + "=" * 80)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print("=" * 80)

# Выводим метрики обучения
print(f"\nФинальные метрики:")
print(f"   Train Loss: {train_result.training_loss:.4f}")
print(f"   Train Runtime: {train_result.metrics['train_runtime']:.2f} секунд")
print(f"   Train Samples/Second: {train_result.metrics['train_samples_per_second']:.2f}")

# Сохраняем финальную модель
trainer.save_model("./models/final")
tokenizer.save_pretrained("./models/final")

print("Модель сохранена")


Старт обучения...


Step,Training Loss,Validation Loss
500,3.362900,3.185733
1000,3.220700,3.058418
1500,3.149700,2.971968
2000,3.032100,2.888084
2500,3.006500,2.835291
3000,2.897000,2.768488
3500,2.889000,2.713115
4000,2.836100,2.671178
4500,2.748400,2.619606
5000,2.763800,2.586241


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].



ОБУЧЕНИЕ ЗАВЕРШЕНО!

Финальные метрики:
   Train Loss: 2.3803
   Train Runtime: 8888.74 секунд
   Train Samples/Second: 17.84
Модель сохранена


In [21]:
# === EVALUATION НА ВАЛИДАЦИОННОМ СЕТЕ ===
print("\nОценка модели на validation set...")
eval_results = trainer.evaluate()

print(f"\nМетрики на validation:")
print(f"   Eval Loss: {eval_results['eval_loss']:.4f}")
print(f"   Perplexity: {np.exp(eval_results['eval_loss']):.2f}")
print(f"   Eval Runtime: {eval_results['eval_runtime']:.2f} секунд")


Оценка модели на validation set...



Метрики на validation:
   Eval Loss: 2.0497
   Perplexity: 7.77
   Eval Runtime: 71.84 секунд


In [22]:
# === ТЕСТИРОВАНИЕ ГЕНЕРАЦИИ ===

def generate_response(context, max_length=50, temperature=0.7, top_p=0.9):
    """
    Генерирует ответ на основе контекста.
    """
    # Форматируем промпт
    prompt = format_prompt(context)
    
    # Токенизируем
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Переносим на нужное устройство
    if device.type != "cpu":
        inputs = {k: v.to(device) for k, v in inputs.items()}
        model.to(device)
    
    # Генерируем
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Декодируем
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Извлекаем только сгенерированную часть (после контекста)
    response = generated_text[len(prompt):].strip()
    
    return response

# Тестируем на примерах из валидации
print("\n=== ПРИМЕРЫ ГЕНЕРАЦИИ ===\n")

test_samples = val_df.sample(5, random_state=42)

for idx, (i, row) in enumerate(test_samples.iterrows(), 1):
    context = row['input']
    true_response = row['output']
    
    generated = generate_response(context, max_length=30, temperature=0.7)
    
    print(f"Пример {idx}:")
    print(f"  Контекст: {context[:100]}...")
    print(f"  Истинный ответ: {true_response}")
    print(f"  Сгенерированный: {generated}")
    print()


=== ПРИМЕРЫ ГЕНЕРАЦИИ ===

Пример 1:
  Контекст: Женщина у психиатра: - Доктор, меня беспокоит мой сын. Он целыми днями лепит куличи из песка и делае...
  Истинный ответ: Ничего страшного, это нормальное поведение ребенка.
  Сгенерированный: Так, так. Сдайте ребенка психиатру. Он должен это все время делать на глазах у жены. И не стесняйтесь, пожалуйста, говорить ему

Пример 2:
  Контекст: Решил Байден посмотреть, как простой народ живет, загримировался, переоделся, пошел на ярмарку. Подх...
  Истинный ответ: Здравствуйте, господин президент
  Сгенерированный: Здравствуйте, я президент секретной службы, вот уже два дня у меня нет посетителей, у меня нет ни одного костюма, а вы просто за мной

Пример 3:
  Контекст: - Мама, а что мы будем праздновать в День отмены рабства? - Это не наш праздник, сынок....
  Истинный ответ: Потому что мы белые?
  Сгенерированный: А мой папа - не наш, он давно умер, и мы не отмечаем его. А мой дедушка - не наш, он тоже давно умер.

Пример 4:
  Контекст: -

In [ ]:
# Примеры для тестирования
test_contexts = [
    "- Почему программисты не любят природу?",
    "Встречаются два друга:\n- Как дела?",
    "Приходит мужик в аптеку и говорит:",
    "- Доктор, у меня проблемы с памятью.",
    "Разговор двух подруг:\n- Как твой новый парень?",
]

print("Примеры контекстов для тестирования:\n")
for i, ctx in enumerate(test_contexts, 1):
    print(f"{i}. {ctx}")

print("\n" + "-" * 80)

# Можно закомментировать для автоматического тестирования
for ctx in test_contexts:
    print(f"\nКонтекст: {ctx}")
    response = generate_response(ctx, max_length=40, temperature=0.8)
    print(f"Ответ модели: {response}")
    print("-" * 80)

Примеры контекстов для тестирования:

1. - Почему программисты не любят природу?
2. Встречаются два друга:
- Как дела?
3. Приходит мужик в аптеку и говорит:
4. - Доктор, у меня проблемы с памятью.
5. Разговор двух подруг:
- Как твой новый парень?

--------------------------------------------------------------------------------

Контекст: - Почему программисты не любят природу?
Ответ модели: Понимаешь, когда ты делаешь что-то, то сразу видно, что это сделано программистом, а когда делаешь что-то, то сразу понимаешь, что это сделано кем-то другим,
--------------------------------------------------------------------------------

Контекст: Встречаются два друга:
- Как дела?
Ответ модели: Да, не спрашивай... У меня теперь есть машина... Тойота... ВАЗ... Мерседес... ВАЗ-2101... Да ну его!.. Иду вчера вечером в вечернюю школу, такая класс
--------------------------------------------------------------------------------

Контекст: Приходит мужик в аптеку и говорит:
Ответ модели: Мне, пожалуйста

In [ ]:
# === ВЫВОДЫ ===

summary = f"""
МОДЕЛЬ ОБУЧЕНА

1. АРХИТЕКТУРА:
   • Модель: {MODEL_NAME}
   • Параметров: {model.num_parameters():,}
   • Размер: {model.num_parameters() * 4 / 1024 / 1024:.1f} MB

2. ПАРАМЕТРЫ ОБУЧЕНИЯ:
   • Epochs: {training_args.num_train_epochs}
   • Batch size: {training_args.per_device_train_batch_size}
   • Learning rate: {training_args.learning_rate}
   • Training samples: {len(train_dataset):,}

3. РЕЗУЛЬТАТЫ:
   • Final Train Loss: {train_result.training_loss:.4f}
   • Validation Loss: {eval_results['eval_loss']:.4f}
   • Perplexity: {np.exp(eval_results['eval_loss']):.2f}
"""

print(summary)


МОДЕЛЬ ОБУЧЕНА

1. АРХИТЕКТУРА:
   • Модель: sberbank-ai/rugpt3small_based_on_gpt2
   • Параметров: 125,226,240
   • Размер: 477.7 MB

2. ПАРАМЕТРЫ ОБУЧЕНИЯ:
   • Epochs: 3
   • Batch size: 8
   • Learning rate: 5e-05
   • Training samples: 52,854

3. РЕЗУЛЬТАТЫ:
   • Final Train Loss: 2.3803
   • Validation Loss: 2.0497
   • Perplexity: 7.77

